In [ ]:
from FMNM.Parameters import Option_param
from FMNM.Processes import Diffusion_process, Merton_process, VG_process, Heston_process
from FMNM.BS_pricer import BS_pricer
from FMNM.Merton_pricer import Merton_pricer
from FMNM.VG_pricer import VG_pricer
from FMNM.Heston_pricer import Heston_pricer

import numpy as np
import pandas as pd
import scipy as scp
import scipy.stats as ss
import matplotlib.pyplot as plt
import scipy.optimize as scpo
from functools import partial
from itertools import compress
import os
import warnings

warnings.filterwarnings("ignore")

In [ ]:
T = 1
S0 = 100
K = 100
opt_param = Option_param(S0=S0, K=K, T=T, v0=0.04, exercise="European", payoff="call")

diff_param = Diffusion_process(r=0.1, sig=0.2)
Merton_param = Merton_process(r=0.1, sig=0.1, lam=0.8, muJ=-0.04, sigJ=0.2)
VG_param = VG_process(r=0.1, theta=-0.09, sigma=0.19, kappa=0.6)
Heston_param = Heston_process(mu=0.1, rho=-0.3, sigma=0.6, theta=0.04, kappa=5)

BS = BS_pricer(opt_param, diff_param)
VG = VG_pricer(opt_param, VG_param)
Mert = Merton_pricer(opt_param, Merton_param)
Hest = Heston_pricer(opt_param, Heston_param)

In [ ]:
strikes = np.arange(50, 151, 5)  # strike grid
BS_prices = BS.FFT(strikes)
Mert_prices = Mert.FFT(strikes)
Hest_prices = Hest.FFT(strikes)
VG_prices = VG.FFT(strikes)

In [ ]:
# Closed formula prices
BS_prices_cl = np.zeros_like(strikes, dtype=float)
VG_prices_cl = np.zeros_like(strikes, dtype=float)
Mert_prices_cl = np.zeros_like(strikes, dtype=float)
Hest_prices_cl = np.zeros_like(strikes, dtype=float)

for i, K in enumerate(strikes):
    BS.K = K
    VG.K = K
    Mert.K = K
    Hest.K = K
    BS_prices_cl[i] = BS.closed_formula()
    VG_prices_cl[i] = VG.Fourier_inversion()
    Mert_prices_cl[i] = Mert.closed_formula()
    Hest_prices_cl[i] = Hest.Fourier_inversion()

In [ ]:
print("Closed vs FFT. Total absolute error BS: ", np.linalg.norm(BS_prices - BS_prices_cl, 1))
print("Closed vs FFT. Total absolute error VG: ", np.linalg.norm(VG_prices - VG_prices_cl, 1))
print("Closed vs FFT. Total absolute error Merton: ", np.linalg.norm(Mert_prices - Mert_prices_cl, 1))
print("Closed vs FFT. Total absolute error Heston: ", np.linalg.norm(Hest_prices - Hest_prices_cl, 1))

In [ ]:
def implied_volatility(price, S0, K, T, r, payoff="call", method="fsolve", disp=True):
    """Returns Implied volatility
    methods:  fsolve (default) or brent
    """

    def obj_fun(vol):
        return price - BS.BlackScholes(payoff=payoff, S0=S0, K=K, T=T, r=r, sigma=vol)

    if method == "brent":
        x, r = scpo.brentq(obj_fun, a=1e-15, b=500, full_output=True)
        if r.converged == True:
            return x
    if method == "fsolve":
        X0 = [0.1, 0.5, 1, 3]  # set of initial guess points
        for x0 in X0:
            x, _, solved, _ = scpo.fsolve(obj_fun, x0, full_output=True, xtol=1e-8)
            if solved == 1:
                return x[0]

    if disp == True:
        print("Strike", K)
    return -1

In [ ]:
def implied_vol_minimize(price, S0, K, T, r, payoff="call", disp=True):
    """Returns Implied volatility by minimization"""

    n = 2  # must be even

    def obj_fun(vol):
        return (BS.BlackScholes(payoff=payoff, S0=S0, K=K, T=T, r=r, sigma=vol) - price) ** n

    res = scpo.minimize_scalar(obj_fun, bounds=(1e-15, 8), method="bounded")
    if res.success == True:
        return res.x
    if disp == True:
        print("Strike", K)
    return -1

In [ ]:
IV_BS = []
IV_VG = []
IV_Mert = []
IV_Hest = []
for i in range(len(strikes)):
    IV_BS.append(implied_volatility(BS_prices[i], S0=100, K=strikes[i], T=T, r=0.1))
    IV_VG.append(implied_volatility(VG_prices[i], S0=100, K=strikes[i], T=T, r=0.1))
    IV_Mert.append(implied_volatility(Mert_prices[i], S0=100, K=strikes[i], T=T, r=0.1))
    IV_Hest.append(implied_volatility(Hest_prices[i], S0=100, K=strikes[i], T=T, r=0.1))

In [ ]:
IV_BS_m = []
IV_VG_m = []
IV_Mert_m = []
IV_Hest_m = []
for i in range(len(strikes)):
    IV_BS_m.append(implied_vol_minimize(BS_prices[i], S0=100, K=strikes[i], T=T, r=0.1))
    IV_VG_m.append(implied_vol_minimize(VG_prices[i], S0=100, K=strikes[i], T=T, r=0.1))
    IV_Mert_m.append(implied_vol_minimize(Mert_prices[i], S0=100, K=strikes[i], T=T, r=0.1))
    IV_Hest_m.append(implied_vol_minimize(Hest_prices[i], S0=100, K=strikes[i], T=T, r=0.1))

In [ ]:
print(
    " Are the IV values obtained by the methods above equal? ",
    np.allclose(np.array(IV_BS), np.array(IV_BS_m))
    & np.allclose(np.array(IV_VG), np.array(IV_VG_m))
    & np.allclose(np.array(IV_Mert), np.array(IV_Mert_m))
    & np.allclose(np.array(IV_Hest), np.array(IV_Hest_m)),
)

In [ ]:
fig = plt.figure(figsize=(16, 4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
ax1.plot(strikes, BS_prices, label="BS")
ax1.plot(strikes, VG_prices, label="VG")
ax1.set_xlim([80, 150])
ax1.set_ylim([0, 30])
ax1.plot(strikes, Mert_prices, label="Mert")
ax1.plot(strikes, Hest_prices, label="Hest")
ax1.set_title("Comparison of prices")
ax1.set_xlabel("Strike")
ax1.set_ylabel("Price")
ax2.plot(strikes, IV_BS, label="BS")
ax2.plot(strikes, IV_VG, label="VG")
ax2.plot(strikes, IV_Mert, label="Mert")
ax2.plot(strikes, IV_Hest, label="Hest")
ax2.set_title("Comparison of Implied volatilities")
ax2.set_xlabel("Strike")
ax2.set_ylabel("Imp Vol")
ax1.legend()
ax2.legend()
plt.show()